In [1]:
import os
import numpy as np
import np_utils
import tensorflow as tf
import pandas as pd
import keras_preprocessing
from tensorflow import keras

In [2]:
from music21 import converter, instrument, note, chord, stream

In [3]:
save_dir = './SingleMidi/'

In [4]:
songList = os.listdir(save_dir)

In [5]:
original_scores = []

In [6]:
for song in songList:
    score = converter.parse(save_dir+song)
    original_scores.append(score)
    print(str(song))

Set_Free_Piano.mid


In [7]:
len(original_scores)

1

In [8]:
original_scores[0].show('midi')

In [9]:
original_scores = [song.chordify() for song in original_scores]

In [10]:
original_chords = [[] for _ in original_scores]
original_durations = [[] for _ in original_scores]
original_keys = []

In [11]:
for i, song in enumerate(original_scores):
    original_keys.append((str(song.analyze('key'))))
    
    for element in song:
        if isinstance(element, note.Note):
            original_chords[i].append(element.pitch)
            original_durations[i].append(element.duration.quarterLength)
        elif isinstance(element, chord.Chord):
            original_chords[i].append('.'.join(str(n) for n in element.pitches))
            original_durations[i].append(element.duration.quarterLength)

In [12]:
unique_chords = np.unique([i for s in original_chords for i in s])
chord_to_int = dict(zip(unique_chords, list(range(0, len(unique_chords)))))

In [13]:
unique_durations = np.unique([i for s in original_durations for i in s])
duration_to_int = dict(zip(unique_durations, list(range(0, len(unique_durations)))))

In [14]:
print(len(unique_chords))
print(len(unique_durations))

30
6


In [15]:
int_to_chord = {i: c for c, i in chord_to_int.items()}
int_to_duration = {i: c for c, i in duration_to_int.items()}

In [16]:
sequenceLength = 32

train_chords = []
train_durations = []


for s in range(len(original_chords)):
    chord_list = [chord_to_int[c] for c in original_chords[s]]
    duration_list = [duration_to_int[d] for d in original_durations[s]]
    
    for i in range(len(chord_list) - sequenceLength):
        train_chords.append(chord_list[i:i+sequenceLength])
        train_durations.append(duration_list[i:i+sequenceLength])

In [17]:
# cMajorChords = [c for (c, k) in zip(original_chords, original_keys) if (k == 'C major')]
# cMajorDurations = [c for (c, k) in zip(original_durations, original_keys) if (k == 'C major')]

In [18]:
# sequenceLength = 32

# train_chords = []
# train_durations = []


# for s in range(len(cMajorChords)):
#     chord_list = [chord_to_int[c] for c in cMajorChords[s]]
#     duration_list = [duration_to_int[d] for d in cMajorDurations[s]]
    
#     for i in range(len(chord_list) - sequenceLength):
#         train_chords.append(chord_list[i:i+sequenceLength])
#         train_durations.append(duration_list[i:i+sequenceLength])

In [19]:
train_chords

[[21,
  19,
  13,
  3,
  10,
  22,
  15,
  25,
  9,
  22,
  15,
  25,
  8,
  21,
  19,
  13,
  3,
  20,
  6,
  29,
  12,
  7,
  5,
  28,
  6,
  2,
  18,
  21,
  19,
  13,
  19,
  24],
 [19,
  13,
  3,
  10,
  22,
  15,
  25,
  9,
  22,
  15,
  25,
  8,
  21,
  19,
  13,
  3,
  20,
  6,
  29,
  12,
  7,
  5,
  28,
  6,
  2,
  18,
  21,
  19,
  13,
  19,
  24,
  17],
 [13,
  3,
  10,
  22,
  15,
  25,
  9,
  22,
  15,
  25,
  8,
  21,
  19,
  13,
  3,
  20,
  6,
  29,
  12,
  7,
  5,
  28,
  6,
  2,
  18,
  21,
  19,
  13,
  19,
  24,
  17,
  27],
 [3,
  10,
  22,
  15,
  25,
  9,
  22,
  15,
  25,
  8,
  21,
  19,
  13,
  3,
  20,
  6,
  29,
  12,
  7,
  5,
  28,
  6,
  2,
  18,
  21,
  19,
  13,
  19,
  24,
  17,
  27,
  25],
 [10,
  22,
  15,
  25,
  9,
  22,
  15,
  25,
  8,
  21,
  19,
  13,
  3,
  20,
  6,
  29,
  12,
  7,
  5,
  28,
  6,
  2,
  18,
  21,
  19,
  13,
  19,
  24,
  17,
  27,
  25,
  1],
 [22,
  15,
  25,
  9,
  22,
  15,
  25,
  8,
  21,
  19,
  13,
  3,
  20,
  6,


In [20]:
from tensorflow.keras.utils import to_categorical

In [21]:
train_chords = np.array(train_chords)

In [22]:
train_chords

array([[21, 19, 13, ..., 13, 19, 24],
       [19, 13,  3, ..., 19, 24, 17],
       [13,  3, 10, ..., 24, 17, 27],
       ...,
       [20,  6, 29, ..., 20, 28, 11],
       [ 6, 29, 12, ..., 28, 11, 29],
       [29, 12,  7, ..., 11, 29,  0]])

In [23]:
train_chords = to_categorical(train_chords).transpose(0, 2, 1)

In [24]:
train_chords.shape

(72, 30, 32)

In [25]:
n_samples = train_chords.shape[0]
n_chords = train_chords.shape[1]
input_dimensions = n_chords * n_samples
latent_dimensions = 2

In [26]:
train_chords_flat = train_chords.flatten()

In [27]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model

In [28]:
encoder_input = Input(shape = (input_dimensions))

In [29]:
latent = Input(shape = (latent_dimensions))

In [30]:
encoded = Dense(latent_dimensions, activation = 'tanh')(encoder_input)

In [31]:
decoded = Dense(input_dimensions, activation = 'sigmoid')(latent)

In [32]:
encoder = Model(encoder_input, encoded)
decoder = Model(latent, decoded)

In [33]:
autoencoder = Model(encoder_input, decoder(encoded))

In [34]:
autoencoder.compile(loss = 'binary_crossentropy', optimizer='rmsprop')

In [35]:
train_chords_reshape = train_chords.reshape(-1, input_dimensions)

In [36]:
autoencoder.fit(train_chords_reshape, train_chords_reshape, epochs=500)

Epoch 1/500
1/1 [==============================] - 1s 590ms/step - loss: 0.6931
Epoch 2/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6909
Epoch 3/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6881
Epoch 4/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6849
Epoch 5/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6818
Epoch 6/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6788
Epoch 7/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6760
Epoch 8/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6735
Epoch 9/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6711
Epoch 10/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6689
Epoch 11/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6668
Epoch 12/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6647
Epoch 13/500
1/1 [==============================] - 0s 3ms/

1/1 [==============================] - 0s 4ms/step - loss: 0.5461
Epoch 105/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5450
Epoch 106/500
1/1 [==============================] - 0s 8ms/step - loss: 0.5438
Epoch 107/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5427
Epoch 108/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5416
Epoch 109/500
1/1 [==============================] - 0s 13ms/step - loss: 0.5405
Epoch 110/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5394
Epoch 111/500
1/1 [==============================] - 0s 5ms/step - loss: 0.5383
Epoch 112/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5372
Epoch 113/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5361
Epoch 114/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5350
Epoch 115/500
1/1 [==============================] - 0s 5ms/step - loss: 0.5339
Epoch 116/500
1/1 [==============================] - 

1/1 [==============================] - 0s 3ms/step - loss: 0.4425
Epoch 207/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4416
Epoch 208/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4406
Epoch 209/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4397
Epoch 210/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4388
Epoch 211/500
1/1 [==============================] - 0s 4ms/step - loss: 0.4379
Epoch 212/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4370
Epoch 213/500
1/1 [==============================] - 0s 4ms/step - loss: 0.4361
Epoch 214/500
1/1 [==============================] - 0s 4ms/step - loss: 0.4352
Epoch 215/500
1/1 [==============================] - 0s 4ms/step - loss: 0.4343
Epoch 216/500
1/1 [==============================] - 0s 4ms/step - loss: 0.4334
Epoch 217/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4326
Epoch 218/500
1/1 [==============================] - 0

1/1 [==============================] - 0s 3ms/step - loss: 0.3593
Epoch 309/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3586
Epoch 310/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3579
Epoch 311/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3572
Epoch 312/500
1/1 [==============================] - 0s 2ms/step - loss: 0.3565
Epoch 313/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3558
Epoch 314/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3550
Epoch 315/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3543
Epoch 316/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3536
Epoch 317/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3529
Epoch 318/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3522
Epoch 319/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3515
Epoch 320/500
1/1 [==============================] - 0

1/1 [==============================] - 0s 4ms/step - loss: 0.2944
Epoch 411/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2938
Epoch 412/500
1/1 [==============================] - 0s 4ms/step - loss: 0.2933
Epoch 413/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2927
Epoch 414/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2922
Epoch 415/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2916
Epoch 416/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2911
Epoch 417/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2905
Epoch 418/500
1/1 [==============================] - 0s 5ms/step - loss: 0.2900
Epoch 419/500
1/1 [==============================] - 0s 4ms/step - loss: 0.2894
Epoch 420/500
1/1 [==============================] - 0s 3ms/step - loss: 0.2889
Epoch 421/500
1/1 [==============================] - 0s 4ms/step - loss: 0.2884
Epoch 422/500
1/1 [==============================] - 0

In [37]:
generated_chords = decoder(np.random.normal(size=(1, latent_dimensions))).numpy().reshape(n_chords, n_samples).argmax(0)

In [38]:
chord_sequence = [int_to_chord[c] for c in generated_chords]

In [39]:
generatedStream = stream.Stream()
generatedStream.append(instrument.Guitar())

# Append notes and chords to stream object
for j in range(len(chord_sequence)):
    try:
        generatedStream.append(note.Note(chord_sequence[j].replace('.', ' ')))
    except:
        generatedStream.append(chord.Chord(chord_sequence[j].replace('.', ' ')))

In [40]:
generatedStream.show('midi')

In [41]:
generatedStream.write('midi')

WindowsPath('C:/Users/adrie/AppData/Local/Temp/music21/tmpnlw4zvbe.mid')